In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI

In [2]:
load_dotenv()  # Load environment variables from .env file

True

In [3]:
model = AzureChatOpenAI(model="gpt-4o-mini")

In [4]:
class BlogState(TypedDict):
    topic: str
    outline: str
    content: str

In [5]:
def generate_outline(state: BlogState) -> BlogState:
    topic = state['topic']
    prompt = f'Generate a detailed outline for a blog post about: {topic}'
    response = model.invoke(prompt).content
    state['outline'] = response
    return state

def generate_content(state: BlogState) -> BlogState:
    outline = state['outline']
    topic = state['topic']
    prompt = f'Write a comprehensive blog post based on the topic: {topic} with following outline:\n{outline}'
    response = model.invoke(prompt).content
    state['content'] = response
    return state

In [7]:
graph = StateGraph(BlogState)

graph.add_node('generate_outline', generate_outline)
graph.add_node('generate_content', generate_content)

graph.add_edge(START, 'generate_outline')
graph.add_edge('generate_outline', 'generate_content')
graph.add_edge('generate_content', END)

workflow=graph.compile()

In [8]:
initial_state={'topic': "The Future of Artificial Intelligence"}
output_state=workflow.invoke(initial_state)
print("Final State:", output_state)

Final State: {'topic': 'The Future of Artificial Intelligence', 'outline': 'Certainly! Here\'s a detailed outline for a blog post titled "The Future of Artificial Intelligence":\n\n---\n\n## Title: The Future of Artificial Intelligence\n\n### Introduction\n- **Hook**: A compelling statistic or quote about the rise of AI technologies.\n- **Purpose of the Post**: Outline the importance of understanding the future trajectory of AI.\n- **Overview**: Briefly introduce key areas that will be explored in the post.\n\n### Section 1: Current Landscape of Artificial Intelligence\n- **1.1 Definition of AI**\n  - Explanation of what artificial intelligence encompasses.\n  - Differences between narrow AI and general AI.\n\n- **1.2 Current Applications**\n  - Overview of where AI is being used today (e.g., healthcare, finance, transportation, customer service).\n  - Examples of successful AI implementations.\n\n- **1.3 Challenges in Current AI**\n  - Discussion of ethical concerns, bias, and data pr